# Excise 2022.11.01




In [1]:
import os
import pyspark
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("test_SamShare").setMaster("local[4]")
sc = SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/01 10:42:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 1. Log file processing


* Input: a simplified log of a web server (i.e., a textual file)

    * Each line of the file is associated with a URL request
    

* Datafile path: ./data/log.txt

In [2]:
#load data
logRDD=sc.textFile('./data/log.txt')
def printRDD(rdd):
    print("Anwser:")
    tmp=rdd.collect()
    for t in tmp:
        print("   ",t)
        
printRDD(logRDD)

Anwser:


    66.249.69.97 - - [24/Sep/2014:22:25:44 +0000] "GET http://www.google.com/bot.html”
    66.249.69.97 - - [24/Sep/2014:22:26:44 +0000] "GET http://www.google.com/how.html”
    66.249.69.97 - - [24/Sep/2014:22:28:44 +0000] "GET http://dbdmg.polito.it/course.html”
    71.19.157.179 - - [24/Sep/2014:22:30:12 +0000] "GET http://www.google.com/faq.html”
    66.249.69.97 - - [24/Sep/2014:31:28:44 +0000] "GET http://dbdmg.polito.it/thesis.html”
    66.249.69.97 - - [24/Sep/2014:56:26:44 +0000] "GET http://www.google.com/how.html”
    56.249.69.97 - - [24/Sep/2014:56:26:44 +0000] "GET http://www.google.com/how.html”


### 1.1  Log filtering

   
* Output: the lines containing the word “google”

    
    
* Answer:
        66.249.69.97 - - [24/Sep/2014:22:25:44 +0000] "GET http://www.google.com/bot.html”
        66.249.69.97 - - [24/Sep/2014:22:26:44 +0000] "GET http://www.google.com/how.html”
        71.19.157.179 - - [24/Sep/2014:22:30:12 +0000] "GET http://www.google.com/faq.html”
        66.249.69.97 - - [24/Sep/2014:56:26:44 +0000] "GET http://www.google.com/how.html”
        56.249.69.97 - - [24/Sep/2014:56:26:44 +0000] "GET http://www.google.com/how.html”
        
Please write your code in the following bracket, and use "shift+return" to check your answer.

In [11]:
# Put your code here:
def contain_google(rdd):
    # tmp = rdd.collect()
    # for line in tmp:
    #     if 'google' in (line):
    #         print(line)

    filter_rdd = rdd.filter(lambda line: 'google' in line)
    #printRDD(rdd)
    return filter_rdd

contain_google(logRDD)           


PythonRDD[2] at RDD at PythonRDD.scala:53

In [3]:
# Put your code here: 

### 1.2 Log analysis

* Output: the list of distinct IP addresses associated with the connections to a google page (i.e., connections to URLs containing the term “www.google.com”)


* Answer:
        66.249.69.97
        71.19.157.179
        56.249.69.97
        
Please write your code in the following bracket, and use "shift+return" to check your answer.

In [13]:
# Put your code here: 
def log_analysis(rdd):
    def get_ip(line):
        return line.split(" ")[0]

    rdd_google = contain_google(rdd)
    
    printRDD(rdd_google.map(get_ip))

log_analysis(logRDD)
    




Anwser:
    66.249.69.97
    66.249.69.97
    71.19.157.179
    66.249.69.97
    56.249.69.97


## 2. Sensor Data Processing

* Input: a collection of (structured) textual csv files containing the daily value of PM10 for a set of sensors
    * Each line of the files has the following format **"sensorId,date,PM10 value (μg/m 3)"**
    

* DataFile Path: ./data/sensor.txt

In [28]:
# put your code here: 
pmrdd = sc.textFile('./data/sensor.txt')
def get_max(rdd):

    rdd = rdd.map(lambda pm: float(pm.split(",")[-1]))
    #rdd.map(kv)
    return rdd.max()

get_max(pmrdd)
    

60.2

### 2.1 Maximum value

* Output: report the maximum value of PM10


* Example:
        60.2
        
Please write your code in the following bracket, and use "shift+return" to check your answer.



In [23]:
### Put your code here: 
def get_max2(rdd):
    def kv(line):
        values = line.split(",")
        return (float(values[-1]),(values[0],values[1]))
    
    rdd = rdd.map(kv)
    rdd = rdd.sortByKey(ascending=True)
    printRDD(rdd)

get_max2(pmrdd)



Anwser:
    (20.4, ('s2', '2016-01-02'))
    (20.5, ('s1', '2016-01-01'))
    (30.1, ('s2', '2016-01-02'))
    (44.7, ('s3', '2016-01-01'))
    (52.5, ('s2', '2016-01-03'))
    (55.5, ('s1', '2016-01-03'))
    (60.2, ('s1', '2016-01-01'))
    (60.2, ('s1', '2016-01-03'))


/opt/miniconda3/envs/myEnv/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:65: UserWarning: Please install psutil to have better support with spilling
/opt/miniconda3/envs/myEnv/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:65: UserWarning: Please install psutil to have better support with spilling


### 2.2 Top-k maximum values

* Output: report the top-3 maximum values of PM10


* set: k=3


* Answer:
        60.2
        60.2
        55.5
        

In [25]:
### Put your code here: 
def get_top3(rdd):

    rdd = rdd.map(lambda pm: pm.split(",")[-1])
    #rdd.map(kv)
    return rdd.top(3)

print(get_top3(pmrdd))

['60.2', '60.2', '55.5']


### 2.3 Readings associated with the maximum value

* Output: the line(s) associated with the maximum value of PM10


* Answer: 
        s1,2016-01-02,60.2 
        s1,2016-01-03,60.2

In [30]:
# put your code here:
def get_max3(rdd):
    def kv(line):
        values = line.split(",")
        return (line,float(values[-1]))

    pair_rdd = rdd.map(kv)
    max_pm = get_max(rdd)
    rdd = pair_rdd.filter(lambda line: line[1] == max_pm)
    #rdd = rdd.sortByKey(ascending=True)
    printRDD(rdd)

get_max3(pmrdd)

Anwser:
    ('s1,2016-01-01,60.2', 60.2)
    ('s1,2016-01-03,60.2', 60.2)


/opt/miniconda3/envs/myEnv/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:65: UserWarning: Please install psutil to have better support with spilling
/opt/miniconda3/envs/myEnv/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:65: UserWarning: Please install psutil to have better support with spilling


### 2.4 Dates associated with the maximum value

* Output: the date(s) associated with the maximum value of PM10


* Answer:
        2016-01-01 
        2016-01-03

In [35]:
# Put your code here: 
def get_max4(rdd):    
    def filter(line):
        if line[1] == max_pm:
            return line[0].split(",")[1]

    max_pm = get_max(rdd)

    rdd = rdd.filter(filter)
    printRDD(rdd)


get_max4(pmrdd)


Anwser:


### 2.5 Average Value

* Output: compute the average PM10 value


* Answer:
        43.0125

In [38]:
# Put your code here: 
def get_mean(rdd):
    return rdd.map(lambda line: float(line.split(",")[-1])).mean()


get_mean(pmrdd)


43.0125

### 2.6 Maximum values

* Output: the maximum value of PM10 for each sensor

* Answer:
        ('s1', 60.2)
        ('s3', 44.7)
        ('s2', 52.5)

In [46]:
# Put your code here:  
def get_max6(rdd):
    def kv(line):
        values = line.split(",")
        return (values[0],float(values[-1]))

    def cmp(d1,d2):
        if d1>d2:
            return d1
        else:
            return d2

    pair_rdd = rdd.map(kv)
    result = pair_rdd.reduceByKey(cmp)#.mapValues(lambda v: max(v, key=lambda x: len(x[1])))
    #rdd = rdd.sortByKey(ascending=True)
    printRDD(result)

get_max6(pmrdd)



Anwser:


/opt/miniconda3/envs/myEnv/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:65: UserWarning: Please install psutil to have better support with spilling
/opt/miniconda3/envs/myEnv/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:65: UserWarning: Please install psutil to have better support with spilling


    ('s1', 60.2)
    ('s3', 44.7)
    ('s2', 52.5)


### 2.7 Pollution analysis

* Output: the sensors with at least 2 readings with a PM10 value greater than the critical threshold 50
    * Store in an HDFS file the sensorIds of the selected sensors and also the number of times each of those sensors is associated with a PM10 value greater than 50
    
    
* Answer:
        (s1,3)

In [53]:
# Put your code here: 
# 首先得到每个sensor种大于50的数量

def test7(rdd):
    def kv(line):
        values = line.split(",")
        return (values[0],float(values[-1]))

    def get_num(line):
        if line[1] > 50:
            return 1
        else:
            return 0

    pair = rdd.map(kv)
    pair = pair.filter(get_num)
    pair = pair.reduceByKey(lambda x,y:x+y)
    pair = pair.filter(lambda line: line[1] >= 2)
    printRDD(pair)

    
test7(pmrdd)




Anwser:
    ('s1', 175.9)
    ('s2', 52.5)


/opt/miniconda3/envs/myEnv/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:65: UserWarning: Please install psutil to have better support with spilling
/opt/miniconda3/envs/myEnv/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:65: UserWarning: Please install psutil to have better support with spilling


### 2.8 Critical dates analysis


* Output: an HDFS file containing one line for each sensor
    * Each line contains a sensorId and the list of dates with a PM10 value greater than 50 for that sensor


* Answer: 
        s1,['2016-01-01', '2016-01-03', '2016-01-03']
        s2,['2016-01-03']

In [55]:
# Put your code here: 
# 首先筛选出大于50
def test8(rdd):
    def kv(line):
        values = line.split(",")
        return (line,float(values[-1]))

    def kv2(line):
        values = line[0].split(",")
        return (values[0], values[1])
    
    

    pair_rdd = rdd.map(kv).filter(lambda line: line[1] > 50)
    pair_rdd = pair_rdd.map(kv2)
    pair_rdd = pair_rdd.groupByKey()

    printRDD(pair_rdd)

    
# 如何打印出？
test8(pmrdd)

Anwser:
    ('s1', <pyspark.resultiterable.ResultIterable object at 0x12f3875e0>)
    ('s2', <pyspark.resultiterable.ResultIterable object at 0x12f425370>)


/opt/miniconda3/envs/myEnv/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:65: UserWarning: Please install psutil to have better support with spilling
/opt/miniconda3/envs/myEnv/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:65: UserWarning: Please install psutil to have better support with spilling


### 2.9  Order sensors by number of critical days

* Output: an HDFS file containing the sensors ordered by the number of critical days

    * Each line of the output file contains the number of days with a PM10 value greater than 50 for a sensor s and the sensorId of sensor
    

* Answer:
        3, s1 
        1, s2

In [58]:
# Put your code here:
def test9(rdd):
    def kv(line):
        values = line.split(",")
        return (line,float(values[-1]))

    def kv2(line):
        values = line[0].split(",")
        return (values[0], values[1])
    
    

    pair_rdd = rdd.map(kv).filter(lambda line: line[1] > 50)
    pair_rdd = pair_rdd.map(kv2)
    pair_rdd = pair_rdd.countByKey()

    print(pair_rdd)

    
# 如何打印出？
test9(pmrdd)



defaultdict(<class 'int'>, {'s1': 3, 's2': 1})


### 2.10 Top-k most critical sensors

* Output:

    * An HDFS file containing the top-k critical sensors

        * The “criticality” of a sensor is given by the number of days with a PM10 value greater than 50 
        * Each line contains the number of critical days and the sensorId
        
        
* set: k=1

* Answer:
        3,S1

In [13]:
# Put your code here: 
def test9(rdd):
    def kv(line):
        values = line.split(",")
        return (line,float(values[-1]))

    def kv2(line):
        values = line[0].split(",")
        return (values[0], values[1])
    
    

    pair_rdd = rdd.map(kv).filter(lambda line: line[1] > 50)
    pair_rdd = pair_rdd.map(kv2)
    pair_rdd = pair_rdd.countByKey()

    print(pair_rdd)

    
# 如何打印出？
test9(pmrdd)










## 3. NLP

* Input:
    * A large textual file containing a set of questions

        * Each line contains one question
            * Each line has the format:
            
                    QuestionId,Timestamp,TextOfTheQuestion
        
    * A large textual file containing a set of answers

        * Each line contains one answer 
        * Each line has the format:
        
                AnswerId,QuestionId,Timestamp,TextOfTheAnswer
                
* Datafile Path: ./data/question.txt, ./data/answer.txt


* Data:
    * Question:
            Q1,2015-01-01,What is ..? 
            Q2,2015-01-03,Who invented ..
            
    * Answer:
            A1,Q1,2015-01-02,It is ..
            A2,Q2,2015-01-03,John Smith
            A3,Q1,2015-01-05,I think it is ..

In [59]:
qRDD=sc.textFile("./data/question.txt")
aRDD=sc.textFile("./data/answer.txt")

### 3.1 Mapping Question-Answer(s)

* Output:

    * A file containing one line for each question 
    * Each line contains a question and the list of answers to that question

            QuestionId, TextOfTheQuestion, list of Answers

In [14]:
# Put your code here 
def test31(rdd1,rdd2):
    def kv1(line):
        return(line.split(",")[0],line.split(",")[-1])

    def kv2(line):
        return(line.split(",")[1],line.split(",")[-1])
    
        
    rdd1 = rdd1.map(kv1)
    rdd2 = rdd2.map(kv2)

    









